###INTRODUÇÃO A INTELIGÊNCIA ARTIFICIAL

Professor: ANDREI DE ARAUJO FORMIGA




###Discentes:

$Arthur\:César$

$Dayane\:Freitas$

$João\:Pedro$

In [7]:
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens

import warnings
warnings.filterwarnings("ignore")

Para visualizações, usamos networkx e matplotlib para mostrar o mapa no notebook e usamos ipywidgets para interagir com o mapa para ver como o algoritmo de busca funciona.


In [8]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

# PROBLEMA


Como definimos um Problema. Execute a próxima célula para ver como a classe abstrata "Problem" é definida no módulo de pesquisa.

In [9]:
class Problem(object):

    def __init__(self, initial, goal=None):
        """O construtor especifica o estado inicial e, possivelmente, um objetivo
        estado, se houver um objetivo único"""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna as ações que podem ser executadas no dado
        Estado"""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna o estado que resulta da execução do dado
        ação no estado dado"""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True se o estado for uma meta. O método padrão compara o
        state para self.goal ou verifica o estado em self.goal se for um
        list, conforme especificado no construtor"""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Retorna o custo de um caminho de solução que chega ao state2 do
        state1 via ação, assumindo custo c para chegar até state1. Se o problema
        é tal que o caminho não importa, esta função irá apenas olhar para
        state2. Se o caminho importa, ele considerará c e talvez state1
        e ação. O método padrão custa 1 para cada passo no caminho."""
        return c + 1

    def value(self, state):
        """Para problemas de otimização, cada estado tem um valor. Escalada
        e algoritmos relacionados tentam maximizar esse valor."""
        raise NotImplementedError

A classe Problem tem seis métodos.

* __init__(self, initial, goal): Isso é o que é chamado de construtor. É o primeiro método chamado quando você cria uma instância da classe como Problem(initial, goal). Uma variável inicial especifica o estado inicial $s_0$ do problema de pesquisa. Representa o estado inicial. A partir daqui, nosso agente inicia sua tarefa de exploração para encontrar o(s) estado(s) objetivo(s) no parâmetro **goal**.


* actions(self, state): Este método retorna todas as ações possíveis que o agente pode executar no estado dado **state**.


*  result(self, state, action): Retorna o estado resultante se a ação **action** for executada no estado** state**. Sendo a classe **Problem** lida apenas com resultados determinísticos. Portanto, sabemos com certeza no que resultaria cada ação em um estado.


*  goal_test(self, state): Retorna um booleano para um determinado estado **True** se for um estado objetivo, senão **False**.


*  path_cost(self, c, state1, action, state2): Retorna o custo do caminho que chega a state2 como resultado de tomar actionde **state1**, assumindo o custo total de cchegar até **state1**.


*  value(self, state): Isso funciona como uma informação extra em problemas em que tentamos otimizar um valor quando não podemos fazer um teste de meta.


# NÓ

Vamos ver como definimos um Nó. Execute a próxima célula para ver como a classe abstrata **Node** é definida no módulo de pesquisa.





In [10]:
class Node:

    """Um nó em uma árvore de pesquisa. Contém um ponteiro para o pai (o nó
    que este é um sucessor de) e para o estado real para este nó. Observação
    que se um estado é alcançado por dois caminhos, então existem dois nós com
    o mesmo estado"""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Cria um nó de árvore de pesquisa, derivado de um pai por uma ação."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """Lista os nós alcançáveis ​​em uma etapa deste nó"""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
      
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Retorna a sequência de ações para ir da raiz até este nó"""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Retorna uma lista de nós que formam o caminho da raiz até este nó"""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

A Nodeclasse tem nove métodos. O primeiro é o __init__método.



*   __init__(self, state, parent, action, path_cost): este método cria um nó. parentrepresenta o nó do qual este é um sucessor e actioné a ação necessária para ir do nó pai para este nó. path_costé o custo para alcançar o nó atual a partir do nó pai.


Os próximos 4 métodos são Nodefunções específicas relacionadas.



*  expand(self, problem): Este método lista todos os nós vizinhos (alcançáveis ​​em uma etapa) do nó atual.


*  child_node(self, problem, action): Dado um action, este método retorna o vizinho imediato que pode ser alcançado com aquele action.



*  solution(self): retorna a sequência de ações necessárias para alcançar este nó a partir do nó raiz.


* path(self): retorna uma lista de todos os nós que estão no caminho da raiz até este nó.



Os 4 métodos restantes substituem a funcionalidade padrão do Python para representar um objeto como uma string, o operador menor que ($<$), o operador igual a ($=$) e a hashfunção.



* __repr__(self): Isso retorna o estado deste nó.


* __lt__(self, node): Dado a node, esse método retorna Truese o estado do nó atual for menor que o estado do node. Caso contrário, ele retorna False.



* __eq__(self, other): Este método retorna Truese o estado do nó atual for igual ao do outro nó. Caso contrário, ele retorna False.



* __hash__(self): Isso retorna o hash do estado do nó atual.


Usaremos a classe abstrata Proble mpara definir nosso problema real chamado GraphProblem.





















In [ ]:
psource(GraphProblem)

In [11]:
class GraphProblem(Problem):

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        Problem.__init__(self, initial, goal)
        self.graph = graph

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get(A).keys())

    def result(self, state, action):
        """The result of going to a neighbor is just that neighbor."""
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or infinity)

    def find_min_edge(self):
        """Find minimum value of edges."""
        m = infinity
        for d in self.graph.graph_dict.values():
            local_min = min(d.values())
            m = min(m, local_min)

        return m

    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(distance(locs[node], locs[self.goal]))

            return int(distance(locs[node.state], locs[self.goal]))
        else:
            return infinity

In [12]:
romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

NameError: name 'UndirectedGraph' is not defined

É bastante simples entender isso romania_map. O primeiro nó Arad tem três vizinhos chamados Zerind , Sibiu , Timisoara . Cada um desses nós tem 75, 140, 118 unidades além de Arad , respectivamente. E o mesmo acontece com outros nós.

E romania_map.locationscontém as posições de cada um dos nós. Usaremos a distância em linha reta (que é diferente da fornecida em romania_map) entre duas cidades em algoritmos como A*-search e Recursive Best First Search.

Definir um problema: Agora é hora de definir nosso problema. Vamos defini-lo passando initial, goal, graphpara GraphProblem. Então, nosso problema é encontrar o estado objetivo a partir do estado inicial dado no gráfico fornecido.

Digamos que queremos começar a explorar a partir de Arad e tentar encontrar Bucareste em nosso romania_map. Então, é assim que fazemos.

In [ ]:
romania_problem = GraphProblem('Arad', 'Bucharest', romania_map)

##PROGRAMA DE AGENTE DE RESOLUÇÃO DE PROBLEMAS SIMPLES


Vamos agora definir um Programa Simples de Agente de Resolução de Problemas. Execute a próxima célula para ver como a classe abstrata SimpleProblemSolvingAgentProgramé definida no módulo de pesquisa.

In [ ]:
psource(SimpleProblemSolvingAgentProgram)

In [ ]:
class SimpleProblemSolvingAgentProgram:

    """Abstract framework for a problem-solving agent. [Figure 3.1]"""

    def __init__(self, initial_state=None):
        """State is an abstract representation of the state
        of the world, and seq is the list of actions required
        to get to a particular state from the initial state(root)."""
        self.state = initial_state
        self.seq = []

    def __call__(self, percept):
        """[Figure 3.1] Formulate a goal and problem, then
        search for a sequence of actions to solve it."""
        self.state = self.update_state(self.state, percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq = self.search(problem)
            if not self.seq:
                return None
        return self.seq.pop(0)

    def update_state(self, state, percept):
        raise NotImplementedError

    def formulate_goal(self, state):
        raise NotImplementedError

    def formulate_problem(self, state, goal):
        raise NotImplementedError

    def search(self, problem):
        raise NotImplementedError

A classe SimpleProblemSolvingAgentProgram tem seis métodos:



* __init__(self, intial_state=None): Este é o contructorda classe e é o primeiro método a ser chamado quando a classe é instanciada. Ele recebe um argumento de palavra-chave, initial_stateque é inicialmente None. O argumento initial_staterepresenta o estado a partir do qual o agente inicia.


* __call__(self, percept): Este método atualiza o stateagente com base em seu perceptuso do update_statemétodo. Em seguida, ele formula a goalcom a ajuda do formulate_goalmétodo e a problemusando o formulate_problemmétodo e retorna uma sequência de ações para resolvê-lo (usando o searchmétodo).


* update_state(self, percept): Este método atualiza o statedo agente com base em seu arquivo percept.


* formulate_goal(self, state): Dado um statedo agente, este método formula o goalpara ele.



* formulate_problem(self, state, goal): É usado na formulação do problema dado a statee a goalpara o agent.


* search(self, problem): Este método é usado para pesquisar uma sequência de actionspara resolver um problem.










Vamos agora definir um Programa Simples de Agente de Resolução de Problemas. Criaremos uma vacuumAgentclasse simples que herdará da classe abstrata SimpleProblemSolvingAgentPrograme substituirá seus métodos. Vamos criar um agente de vácuo inteligente simples que pode estar em qualquer um dos seguintes estados. Ele se moverá para qualquer outro estado, dependendo do estado atual, conforme mostrado na imagem por setas:

In [ ]:
class vacuumAgent(SimpleProblemSolvingAgentProgram):
        def update_state(self, state, percept):
            return percept

        def formulate_goal(self, state):
            goal = [state7, state8]
            return goal  

        def formulate_problem(self, state, goal):
            problem = state
            return problem   
    
        def search(self, problem):
            if problem == state1:
                seq = ["Suck", "Right", "Suck"]
            elif problem == state2:
                seq = ["Suck", "Left", "Suck"]
            elif problem == state3:
                seq = ["Right", "Suck"]
            elif problem == state4:
                seq = ["Suck"]
            elif problem == state5:
                seq = ["Suck"]
            elif problem == state6:
                seq = ["Left", "Suck"]
            return seq

Agora, vamos definir todos os 8 estados e criar um objeto da classe acima. Então, vamos passar diferentes estados e verificar a saída:

In [ ]:
state1 = [(0, 0), [(0, 0), "Dirty"], [(1, 0), ["Dirty"]]]
state2 = [(1, 0), [(0, 0), "Dirty"], [(1, 0), ["Dirty"]]]
state3 = [(0, 0), [(0, 0), "Clean"], [(1, 0), ["Dirty"]]]
state4 = [(1, 0), [(0, 0), "Clean"], [(1, 0), ["Dirty"]]]
state5 = [(0, 0), [(0, 0), "Dirty"], [(1, 0), ["Clean"]]]
state6 = [(1, 0), [(0, 0), "Dirty"], [(1, 0), ["Clean"]]]
state7 = [(0, 0), [(0, 0), "Clean"], [(1, 0), ["Clean"]]]
state8 = [(1, 0), [(0, 0), "Clean"], [(1, 0), ["Clean"]]]

a = vacuumAgent(state1)

print(a(state6)) 
print(a(state1))
print(a(state3))

##Pesquisa em profundidade

In [ ]:
def tree_depth_search_for_vis(problem):
    """Pesquisa os sucessores de um problema para encontrar um objetivo.
    A fronteira do argumento deve ser uma fila vazia."""
    
    
    iterations = 0
    all_node_colors = []
    node_colors = {k : 'white' for k in problem.graph.nodes()}
    
   
    frontier = [Node(problem.initial)]
    
    node_colors[Node(problem.initial).state] = "orange"
    iterations += 1
    all_node_colors.append(dict(node_colors))
    
    while frontier:
     
        node = frontier.pop()
       
        node_colors[node.state] = "red"
        iterations += 1
        all_node_colors.append(dict(node_colors))
        
        if problem.goal_test(node.state):
      
            node_colors[node.state] = "green"
            iterations += 1
            all_node_colors.append(dict(node_colors))
            return(iterations, all_node_colors, node)
        
        frontier.extend(node.expand(problem))
           
        for n in node.expand(problem):
            node_colors[n.state] = "orange"
            iterations += 1
            all_node_colors.append(dict(node_colors))

      
        node_colors[node.state] = "gray"
        iterations += 1
        all_node_colors.append(dict(node_colors))
        
    return None

def depth_first_tree_search(problem):
    "Search the deepest nodes in the search tree first."
    iterations, all_node_colors, node = tree_depth_search_for_vis(problem)
    return(iterations, all_node_colors, node)

##Problema de quebra-cabeça

O Problema do 8 Puzzle consiste em uma bandeja 3x3 na qual o objetivo é obter a configuração inicial para o estado objetivo, deslocando as peças numeradas para o espaço em branco.

Temos um total de 9 peças em branco, dando-nos um total de 9! configuração inicial, mas nem todas são solucionáveis. A solubilidade de uma configuração pode ser verificada calculando a Permutação de Inversão. Se a Permutação de Inversão total for par, então a configuração inicial é solucionável, caso contrário a configuração inicial não é solucionável, o que significa que apenas 9!/2 estados iniciais levam a uma solução.
Vamos definir nosso estado de meta.

In [ ]:
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

In [ ]:

def linear(node):
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd
    
    return mhd

def sqrt_manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = (index_goal[i][j] - index_state[i][j])**2 + mhd
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

Podemos resolver o quebra-cabeça usando o método astar_search.

In [ ]:
puzzle = EightPuzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle.check_solvability((2, 4, 3, 1, 5, 6, 7, 8, 0))

Este caso é solucionável, vamos prosseguir.
A função heurística padrão retorna o número de ladrilhos mal colocados.

In [ ]:
astar_search(puzzle).solution()

Nas células a seguir, usamos diferentes funções heurísticas.

In [ ]:
astar_search(puzzle, linear).solution()

In [ ]:
astar_search(puzzle, manhattan).solution()

In [ ]:
astar_search(puzzle, max_heuristic).solution()

E aqui está como recursive_best_first_searchpode ser usado para resolver este problema também.

In [ ]:
recursive_best_first_search(puzzle, manhattan).solution()

Mesmo que todas as funções heurísticas forneçam a mesma solução, a diferença está no tempo de computação.
Isso pode fazer toda a diferença em um cenário onde é necessária alta eficiência computacional.
Vamos definir alguns estados de quebra-cabeça e tempo astar_searchpara cada função heurística. Usaremos a mágica %%timeit para isso.

In [ ]:
puzzle_1 = EightPuzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = EightPuzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = EightPuzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

A função heurística padrão é a mesma que a linearfunção heurística, mas ainda verificaremos ambas.

In [ ]:
%%timeit
astar_search(puzzle_1)
astar_search(puzzle_2)
astar_search(puzzle_3)

In [ ]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

In [ ]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

Podemos inferir que a manhattanfunção heurística funciona mais rápido.
sqrt_manhattantem uma operação extra sqrtque o torna muito mais lento que os outros.
max_heuristicdeveria ter sido um pouco mais lento, pois chama duas funções, mas neste caso, esses valores já foram calculados o que economizou algum tempo. Sinta-se à vontade para brincar com essas funções.

Para comparação, é assim que o RBFS se comporta nesse problema.

In [ ]:
%%timeit
recursive_best_first_search(puzzle_1, linear)
recursive_best_first_search(puzzle_2, linear)
recursive_best_first_search(puzzle_3, linear)

É muito mais lento do astar_searchque podemos ver.